In [1]:
from astropy import units as u
import numpy as np
from astromatch import Catalogue
from mocpy import MOC
from astromatch import Match
from astromatch.priorsND import PriorND
import sys


This examples uses a source catalogue from the XMM-ATLAS survey (http://xraygroup.astro.noa.gr/atlas.html). The used catalogue is the result of custom X-ray data analysis and does not correspond to the one provided  in the lnk above The catalogue is uploaded into the astromatch Catalogue class. The corresponding MOC that describes the XMM-ATLAS field of view is also given. 

In [2]:
mocx = MOC.from_fits("DATA/xmmatlas_new.moc")     
xcat_table="DATA/xmmatlas_srclistnew_Legacy.fits"
xcat = Catalogue(
    xcat_table,
    name='xatlas',
    id_col='UXID',
    coord_cols=['RA_CORR', 'DEC_CORR'],
    poserr_cols=['RADEC_TERR'],
    poserr_type='circle',
    area=mocx,
)


The next steps is to define the optical catalogue and pass intothe astromatch Catalogue class. In this case the Legacy survey (http://legacysurvey.org/) dataset is used. The table columns of the Legacy survey source list that will be read into the Catalogue class need to be defined. These are the columns that will be used for the construction of priors. We therefore define the prios via the mags variable here. The construction of priors also require boundaries and bin sizes to construct the corresponding N-dimensional histograms. There are also defined here. At this stage no prior is contructed by the relevant columns are read in for the calculations later on. 

In [3]:
moco = MOC.from_fits("DATA/legacy.moc")
mags = [['MAG_G', 'MAG_Z'], ['MAG_W1'],['MAG_W2'], ['GAIA_PHOT_G_MEAN_MAG']]
magmin = [ [10, 10] ,[10, 10], [10], [10], [5]]
magmax = [ [30, 30], [30, 30], [30], [30], [25]]
magbinsize = [ [0.25, 0.5], [0.25, 0.5], [0.5], [0.5]]

ocat = Catalogue(
    "DATA/Legacy_SLIM.fits",
    name='lgcy',
    id_col='UOBJID',
    coord_cols=['RA', 'DEC'],
    poserr_cols=['RADEC_ERR'],
    poserr_type='circle',
    area=moco,
    mag_cols=mags,
)

Define the Match object of the two catalogues. 



In [4]:
xm = Match(xcat, ocat)

The two catalogue are matched. The priors are also constructed. With the definitions above they consist of one 2-d prior from the  MAG_G and MAG_Z columns in the  DATA/Legacy_SLIM.fits file and three 1-D priors using the columns  'MAG_W1', 'MAG_W2', 'GAIA_PHOT_G_MEAN_MAG' in the same file. The prior is determined using the "random" method using a total of 100 random samples. The Likelihood Ratio (lr) method is used to determine the probability of sources being associated. The positional offset probability distribution is described by the Rayleigh distribution with sigma the combined positional uncertainty of the sources in each catalogue.  

In [5]:
match_results = xm.run(method='lr', radius=7.5*u.arcsec, prior_method='random', 
                       mags = mags, magmin=magmin, magmax=magmax, 
                       magbinsize=magbinsize,random_numrepeat=100, 
                       poserr_dist="rayleigh")

INFO: Using LR method: [astromatch.core]
INFO: Searching for match candidates within 7.5 arcsec... [astromatch.lr]
INFO: Calculating priors... [astromatch.lr]
INFO: Estinating priors from Catalogue using columns [['MAG_G', 'MAG_Z'], ['MAG_W1'], ['MAG_W2'], ['GAIA_PHOT_G_MEAN_MAG']]... [astromatch.priorsND]
INFO: Calculating likelihood ratios for match candidates... [astromatch.lr]
[0.9531438767546949, 0.988280739854066, 0.9407345573195783, 0.3441023670731921]
INFO: Sorting and flagging match results... [astromatch.lr]


/soft9/anaconda3/envs/python3/lib/python3.7/site-packages/astropy/units/quantity.py:477: RuntimeWarning: invalid value encountered in true_divide
  result = super().__array_ufunc__(function, method, *arrays, **kwargs)
/soft9/anaconda3/envs/python3/lib/python3.7/site-packages/astromatch/lr.py:495: RuntimeWarning: invalid value encountered in true_divide
  lr_table['p_any_' + col])
/soft9/anaconda3/envs/python3/lib/python3.7/site-packages/astromatch/lr.py:728: RuntimeWarning: invalid value encountered in true_divide
  mask_ratio = match['prob_this_match']/pi_max > prob_ratio_secondary
/soft9/anaconda3/envs/python3/lib/python3.7/site-packages/astropy/table/column.py:1020: RuntimeWarning: invalid value encountered in greater
  result = getattr(super(), op)(other)


it is possible to visualse the prior by saving it into a fits file

In [6]:
priors=xm._match.priors
t = priors.to_table(include_bkg_priors=False)
t.writeto('DATA/prior.fits', overwrite=True)

the same for background density of source

In [7]:
bkg=xm._match._bkg
t = bkg.to_table()
t.writeto('DATA/bkg.fits', overwrite=True)

Define a ploting function for visualising thre results.

In [8]:
def plot_stats(stats):
        
    import matplotlib.pyplot as plt
    #from matplotlib.backends.backend_pdf import PdfPages
    from matplotlib.ticker import MultipleLocator, FormatStrFormatter
    import os
    
    #plotfile = os.path.join('./stats-{}.pdf'.format(ext))
          
    fontPanel = {'family': 'serif',
                     'color': 'black',
                     'weight': 'bold',
                     'size': 10,} 

    figall=[]
    pdf = PdfPages(plotfile)
    fig, ax = plt.subplots(1,1)
    y= stats['completeness']
    x= stats['cutoff']
    ax.plot(x,y, 'r-', label="completeness")            
    y= stats['reliability']
    ax.plot(x,y, 'b--', label="purity")            
    
    for axis in ['top','bottom','left','right']:
        for a in [ax]:
            a.spines[axis].set_linewidth(2)
            a.tick_params(labelsize=12)
            a.xaxis.set_tick_params(which='major', width=1.5, size=8)
            a.yaxis.set_tick_params(which='major', width=1.5, size=8)
            a.xaxis.set_tick_params(which='minor', width=1.5, size=5)
            a.yaxis.set_tick_params(which='minor', width=1.5, size=5)
         
    xmax=3
    m= stats['cutoff']<xmax
    min1 = np.min(stats['completeness'][m])
    min2 = np.min(stats['reliability'][m])
    min1 = min(min1,min2)
    max1 = np.max(stats['completeness'][m])
    max2 = np.max(stats['completeness'][m])
    max1 = max(max1,max2)
    
    ax.set_xlim(0,xmax)
    ax.set_ylim(min1*0.95, max1*1.05)
    ax.grid()
    ax.legend()
    plt.show()
    #pdf.savefig( fig )
    #pdf.close()
 


In [9]:
xm.set_best_matchs(cutoff=-9.)
lr_stats = xm.stats(ncutoff=501, maxcutoff=10.0)
lr_stats.write("stats.fits", format='fits', overwrite=True)
plot_stats(stats=lr_stats)

/soft9/anaconda3/envs/python3/lib/python3.7/site-packages/astropy/table/column.py:984: UserWarning: Warning: converting a masked element to nan.
  self.data[index] = value


NameError: name 'PdfPages' is not defined